# Visualize Trained Agents

In [1]:
from orbax.checkpoint import PyTreeCheckpointer

import _pickle as pickle
import jax
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import timeit

from jaxl.buffers import get_buffer
from jaxl.constants import *
from jaxl.models import (
    get_model,
    get_policy,
    policy_output_dim,
)
from jaxl.models.policies import MultitaskPolicy
from jaxl.envs import get_environment
from jaxl.envs.rollouts import EvaluationRollout
from jaxl.utils import set_seed, parse_dict

In [2]:
run_seed = None
set_seed(run_seed)

In [3]:
rl_trained_path = "/home/chanb/Documents/research/personal/jaxl/jaxl/logs/inverted_double_pendulum/ppo/06-07-23_18_51_32-7a91e450-3b0a-40f1-9b45-e39f10be0b29"

num_episodes = 10
env_seed = 9999
buffer_size = 0

In [4]:
rl_config_path = os.path.join(rl_trained_path, "config.json")
with open(rl_config_path, "r") as f:
    rl_config_dict = json.load(f)
    rl_config_dict["learner_config"]["env_config"]["env_kwargs"][
        "render_mode"
    ] = "rgb_array"
    rl_config_dict["learner_config"]["env_config"]["env_kwargs"][
        "gravity"
    ] = -1.0
    rl_config = parse_dict(rl_config_dict)

In [5]:
from gymnasium.experimental.wrappers import RecordVideoV0

In [6]:
h_state_dim = (1,)
if hasattr(rl_config.model_config, "h_state_dim"):
    h_state_dim = rl_config.model_config.h_state_dim

env = get_environment(rl_config.learner_config.env_config)
env = RecordVideoV0(env, "videos", episode_trigger=lambda x: True)

input_dim = env.observation_space.shape
output_dim = policy_output_dim(env.action_space.shape, rl_config.learner_config)
model = get_model(input_dim, output_dim, rl_config.model_config.policy)
policy = get_policy(model, rl_config.learner_config)

rl_model_path = os.path.join(rl_trained_path, "termination_model")
checkpointer = PyTreeCheckpointer()
model_dict = checkpointer.restore(rl_model_path)
rl_policy_params = model_dict[CONST_MODEL][CONST_POLICY]
with open(os.path.join(rl_model_path, "learner_dict.pkl"), "rb") as f:
    learner_dict = pickle.load(f)
    rl_obs_rms = learner_dict[CONST_OBS_RMS]

rl_rollout = EvaluationRollout(env, seed=env_seed)
rl_rollout.rollout(rl_policy_params, policy, rl_obs_rms, num_episodes, None)

/home/chanb/Documents/research/personal/jaxl/.venv/lib/python3.9/site-packages/gymnasium/experimental/wrappers/rendering.py:169: UserWarning: WARN: Overwriting existing videos at /home/chanb/Documents/research/personal/jaxl/notebooks/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


/tmp/tmp6q174xs_
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-0.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-1.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-2.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-2.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-3.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-3.mp4



...

Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-4.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-5.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-5.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-5.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-6.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-6.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-6.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-7.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-7.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-7.mp4
Moviepy - Building video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-8.mp4.
Moviepy - Writing video /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /home/chanb/Documents/research/personal/jaxl/notebooks/videos/rl-video-episode-8.mp4
